# 국면예측작업(예측 모델 작업에서 선택된 모델을 바탕으로 현실 데이터를 받아와서 향후 3개월 후를 예측)

In [ ]:
# 드라이브에 마운트
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/project

Mounted at /content/drive
/content/drive/MyDrive/project


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib import font_manager
import seaborn as sns
plt.rc('font', family='NanumBarunGothic')
# plt.rcParams['font.family'] = font_manager.FontProperties(fname=font_path).get_name()
# plt.rcParams['axes.unicode_minus'] = False
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 18, 6


In [ ]:
df_HG1 = pd.read_csv('Copper Futures Historical Data.csv')
df_HG1

,Date,Price,Open,High,Low,Vol.,Change %
0,10/31/2023,3.6490,3.6550,3.6745,3.6315,72.26K,-0.26%
1,10/30/2023,3.6585,3.6425,3.7100,3.6400,93.45K,0.34%
2,10/27/2023,3.6460,3.5935,3.6595,3.5810,102.14K,1.72%
3,10/26/2023,3.5845,3.5890,3.6195,3.5720,73.83K,-0.18%
4,10/25/2023,3.5910,3.6315,3.6490,3.5820,70.33K,-0.91%
...,...,...,...,...,...,...,...
62,08/07/2023,3.8350,3.8520,3.8770,3.8250,70.87K,-0.84%
63,08/04/2023,3.8675,3.8995,3.9280,3.8255,79.91K,-0.82%
64,08/03/2023,3.8995,3.8445,3.9040,3.8205,81.96K,1.46%
65,08/02/2023,3.8435,3.9100,3.9210,3.8340,89.27K,-1.66%


In [ ]:
df_HG1['date'] = df_HG1['Date'].apply(lambda x: pd.to_datetime(str(x), format='%m/%d/%Y')) # 시계열 정보 추출
df_HG1

,Date,Price,Open,High,Low,Vol.,Change %,date
0,10/31/2023,3.6490,3.6550,3.6745,3.6315,72.26K,-0.26%,2023-10-31
1,10/30/2023,3.6585,3.6425,3.7100,3.6400,93.45K,0.34%,2023-10-30
2,10/27/2023,3.6460,3.5935,3.6595,3.5810,102.14K,1.72%,2023-10-27
3,10/26/2023,3.5845,3.5890,3.6195,3.5720,73.83K,-0.18%,2023-10-26
4,10/25/2023,3.5910,3.6315,3.6490,3.5820,70.33K,-0.91%,2023-10-25
...,...,...,...,...,...,...,...,...
62,08/07/2023,3.8350,3.8520,3.8770,3.8250,70.87K,-0.84%,2023-08-07
63,08/04/2023,3.8675,3.8995,3.9280,3.8255,79.91K,-0.82%,2023-08-04
64,08/03/2023,3.8995,3.8445,3.9040,3.8205,81.96K,1.46%,2023-08-03
65,08/02/2023,3.8435,3.9100,3.9210,3.8340,89.27K,-1.66%,2023-08-02


In [ ]:
df_HG1['ticker1'] = 'TK_HG1 Comdty'
df_HG1

,Date,Price,Open,High,Low,Vol.,Change %,date,ticker1
0,10/31/2023,3.6490,3.6550,3.6745,3.6315,72.26K,-0.26%,2023-10-31,TK_HG1 Comdty
1,10/30/2023,3.6585,3.6425,3.7100,3.6400,93.45K,0.34%,2023-10-30,TK_HG1 Comdty
2,10/27/2023,3.6460,3.5935,3.6595,3.5810,102.14K,1.72%,2023-10-27,TK_HG1 Comdty
3,10/26/2023,3.5845,3.5890,3.6195,3.5720,73.83K,-0.18%,2023-10-26,TK_HG1 Comdty
4,10/25/2023,3.5910,3.6315,3.6490,3.5820,70.33K,-0.91%,2023-10-25,TK_HG1 Comdty
...,...,...,...,...,...,...,...,...,...
62,08/07/2023,3.8350,3.8520,3.8770,3.8250,70.87K,-0.84%,2023-08-07,TK_HG1 Comdty
63,08/04/2023,3.8675,3.8995,3.9280,3.8255,79.91K,-0.82%,2023-08-04,TK_HG1 Comdty
64,08/03/2023,3.8995,3.8445,3.9040,3.8205,81.96K,1.46%,2023-08-03,TK_HG1 Comdty
65,08/02/2023,3.8435,3.9100,3.9210,3.8340,89.27K,-1.66%,2023-08-02,TK_HG1 Comdty


In [ ]:
df_CL1 = pd.read_csv('Crude Oil WTI Futures Historical Data.csv')
df_CL1

,Date,Price,Open,High,Low,Vol.,Change %
0,10/31/2023,81.02,82.59,83.37,80.74,346.01K,-1.57%
1,10/30/2023,82.31,85.00,85.30,81.82,356.93K,-3.78%
2,10/27/2023,85.54,83.53,85.90,83.10,333.76K,2.80%
3,10/26/2023,83.21,85.37,85.59,82.56,341.79K,-2.55%
4,10/25/2023,85.39,83.69,85.56,82.08,429.46K,1.97%
...,...,...,...,...,...,...,...
62,08/07/2023,81.94,82.82,83.30,81.52,293.04K,-1.06%
63,08/04/2023,82.82,81.73,83.24,81.51,320.87K,1.56%
64,08/03/2023,81.55,79.77,81.86,78.69,402.64K,2.59%
65,08/02/2023,79.49,82.05,82.43,79.05,414.83K,-2.31%


In [ ]:
df_CL1['date'] = df_CL1['Date'].apply(lambda x: pd.to_datetime(str(x), format='%m/%d/%Y')) # 시계열 정보 추출
df_CL1

,Date,Price,Open,High,Low,Vol.,Change %,date
0,10/31/2023,81.02,82.59,83.37,80.74,346.01K,-1.57%,2023-10-31
1,10/30/2023,82.31,85.00,85.30,81.82,356.93K,-3.78%,2023-10-30
2,10/27/2023,85.54,83.53,85.90,83.10,333.76K,2.80%,2023-10-27
3,10/26/2023,83.21,85.37,85.59,82.56,341.79K,-2.55%,2023-10-26
4,10/25/2023,85.39,83.69,85.56,82.08,429.46K,1.97%,2023-10-25
...,...,...,...,...,...,...,...,...
62,08/07/2023,81.94,82.82,83.30,81.52,293.04K,-1.06%,2023-08-07
63,08/04/2023,82.82,81.73,83.24,81.51,320.87K,1.56%,2023-08-04
64,08/03/2023,81.55,79.77,81.86,78.69,402.64K,2.59%,2023-08-03
65,08/02/2023,79.49,82.05,82.43,79.05,414.83K,-2.31%,2023-08-02


In [ ]:
df_CL1['ticker1'] = 'TK_CL1 Comdty'
df_CL1

,Date,Price,Open,High,Low,Vol.,Change %,date,ticker1
0,10/31/2023,81.02,82.59,83.37,80.74,346.01K,-1.57%,2023-10-31,TK_CL1 Comdty
1,10/30/2023,82.31,85.00,85.30,81.82,356.93K,-3.78%,2023-10-30,TK_CL1 Comdty
2,10/27/2023,85.54,83.53,85.90,83.10,333.76K,2.80%,2023-10-27,TK_CL1 Comdty
3,10/26/2023,83.21,85.37,85.59,82.56,341.79K,-2.55%,2023-10-26,TK_CL1 Comdty
4,10/25/2023,85.39,83.69,85.56,82.08,429.46K,1.97%,2023-10-25,TK_CL1 Comdty
...,...,...,...,...,...,...,...,...,...
62,08/07/2023,81.94,82.82,83.30,81.52,293.04K,-1.06%,2023-08-07,TK_CL1 Comdty
63,08/04/2023,82.82,81.73,83.24,81.51,320.87K,1.56%,2023-08-04,TK_CL1 Comdty
64,08/03/2023,81.55,79.77,81.86,78.69,402.64K,2.59%,2023-08-03,TK_CL1 Comdty
65,08/02/2023,79.49,82.05,82.43,79.05,414.83K,-2.31%,2023-08-02,TK_CL1 Comdty


In [ ]:
USDKRW = pd.read_csv('USD_KRW Historical Data.csv')
USDKRW

,Date,Price,Open,High,Low,Vol.,Change %
0,10/31/2023,"1,352.15","1,346.49","1,356.01","1,345.26",NaN,0.48%
1,10/30/2023,"1,345.63","1,357.05","1,357.45","1,345.52",NaN,-0.68%
2,10/27/2023,"1,354.91","1,353.57","1,358.34","1,347.76",NaN,0.16%
3,10/26/2023,"1,352.69","1,352.90","1,360.72","1,351.70",NaN,0.02%
4,10/25/2023,"1,352.44","1,344.34","1,354.15","1,343.22",NaN,0.64%
...,...,...,...,...,...,...,...
61,08/07/2023,"1,307.30","1,304.29","1,309.06","1,298.80",NaN,0.25%
62,08/04/2023,"1,304.08","1,299.93","1,312.38","1,296.82",NaN,0.39%
63,08/03/2023,"1,299.01","1,299.47","1,304.23","1,294.55",NaN,0.01%
64,08/02/2023,"1,298.93","1,289.40","1,303.09","1,286.94",NaN,0.77%


In [ ]:
USDKRW['date'] = USDKRW['Date'].apply(lambda x: pd.to_datetime(str(x), format='%m/%d/%Y')) # 시계열 정보 추출
USDKRW

,Date,Price,Open,High,Low,Vol.,Change %,date
0,10/31/2023,"1,352.15","1,346.49","1,356.01","1,345.26",NaN,0.48%,2023-10-31
1,10/30/2023,"1,345.63","1,357.05","1,357.45","1,345.52",NaN,-0.68%,2023-10-30
2,10/27/2023,"1,354.91","1,353.57","1,358.34","1,347.76",NaN,0.16%,2023-10-27
3,10/26/2023,"1,352.69","1,352.90","1,360.72","1,351.70",NaN,0.02%,2023-10-26
4,10/25/2023,"1,352.44","1,344.34","1,354.15","1,343.22",NaN,0.64%,2023-10-25
...,...,...,...,...,...,...,...,...
61,08/07/2023,"1,307.30","1,304.29","1,309.06","1,298.80",NaN,0.25%,2023-08-07
62,08/04/2023,"1,304.08","1,299.93","1,312.38","1,296.82",NaN,0.39%,2023-08-04
63,08/03/2023,"1,299.01","1,299.47","1,304.23","1,294.55",NaN,0.01%,2023-08-03
64,08/02/2023,"1,298.93","1,289.40","1,303.09","1,286.94",NaN,0.77%,2023-08-02


In [ ]:
USDKRW['ticker1'] = 'TK_USDKRW BGN Curncy'
USDKRW

,Date,Price,Open,High,Low,Vol.,Change %,date,ticker1
0,10/31/2023,"1,352.15","1,346.49","1,356.01","1,345.26",NaN,0.48%,2023-10-31,TK_USDKRW BGN Curncy
1,10/30/2023,"1,345.63","1,357.05","1,357.45","1,345.52",NaN,-0.68%,2023-10-30,TK_USDKRW BGN Curncy
2,10/27/2023,"1,354.91","1,353.57","1,358.34","1,347.76",NaN,0.16%,2023-10-27,TK_USDKRW BGN Curncy
3,10/26/2023,"1,352.69","1,352.90","1,360.72","1,351.70",NaN,0.02%,2023-10-26,TK_USDKRW BGN Curncy
4,10/25/2023,"1,352.44","1,344.34","1,354.15","1,343.22",NaN,0.64%,2023-10-25,TK_USDKRW BGN Curncy
...,...,...,...,...,...,...,...,...,...
61,08/07/2023,"1,307.30","1,304.29","1,309.06","1,298.80",NaN,0.25%,2023-08-07,TK_USDKRW BGN Curncy
62,08/04/2023,"1,304.08","1,299.93","1,312.38","1,296.82",NaN,0.39%,2023-08-04,TK_USDKRW BGN Curncy
63,08/03/2023,"1,299.01","1,299.47","1,304.23","1,294.55",NaN,0.01%,2023-08-03,TK_USDKRW BGN Curncy
64,08/02/2023,"1,298.93","1,289.40","1,303.09","1,286.94",NaN,0.77%,2023-08-02,TK_USDKRW BGN Curncy


In [ ]:
USDJPY = pd.read_csv('USD_JPY Historical Data.csv')
USDJPY

,Date,Price,Open,High,Low,Vol.,Change %
0,10/31/2023,151.67,149.10,151.76,149.02,NaN,1.72%
1,10/30/2023,149.10,149.54,149.93,148.80,NaN,-0.33%
2,10/27/2023,149.60,150.38,150.43,149.46,NaN,-0.52%
3,10/26/2023,150.38,150.21,150.79,149.90,NaN,0.11%
4,10/25/2023,150.21,149.91,150.32,149.79,NaN,0.20%
...,...,...,...,...,...,...,...
61,08/07/2023,142.49,141.79,142.62,141.52,NaN,0.52%
62,08/04/2023,141.75,142.57,142.89,141.55,NaN,-0.55%
63,08/03/2023,142.54,143.33,143.90,142.06,NaN,-0.54%
64,08/02/2023,143.32,143.34,143.49,142.24,NaN,0.00%


In [ ]:
USDJPY['date'] = USDJPY['Date'].apply(lambda x: pd.to_datetime(str(x), format='%m/%d/%Y')) # 시계열 정보 추출
USDJPY

,Date,Price,Open,High,Low,Vol.,Change %,date
0,10/31/2023,151.67,149.10,151.76,149.02,NaN,1.72%,2023-10-31
1,10/30/2023,149.10,149.54,149.93,148.80,NaN,-0.33%,2023-10-30
2,10/27/2023,149.60,150.38,150.43,149.46,NaN,-0.52%,2023-10-27
3,10/26/2023,150.38,150.21,150.79,149.90,NaN,0.11%,2023-10-26
4,10/25/2023,150.21,149.91,150.32,149.79,NaN,0.20%,2023-10-25
...,...,...,...,...,...,...,...,...
61,08/07/2023,142.49,141.79,142.62,141.52,NaN,0.52%,2023-08-07
62,08/04/2023,141.75,142.57,142.89,141.55,NaN,-0.55%,2023-08-04
63,08/03/2023,142.54,143.33,143.90,142.06,NaN,-0.54%,2023-08-03
64,08/02/2023,143.32,143.34,143.49,142.24,NaN,0.00%,2023-08-02


In [ ]:
USDJPY['ticker1'] = 'TK_USDJPY BGN Curncy'
USDJPY

,Date,Price,Open,High,Low,Vol.,Change %,date,ticker1
0,10/31/2023,151.67,149.10,151.76,149.02,NaN,1.72%,2023-10-31,TK_USDJPY BGN Curncy
1,10/30/2023,149.10,149.54,149.93,148.80,NaN,-0.33%,2023-10-30,TK_USDJPY BGN Curncy
2,10/27/2023,149.60,150.38,150.43,149.46,NaN,-0.52%,2023-10-27,TK_USDJPY BGN Curncy
3,10/26/2023,150.38,150.21,150.79,149.90,NaN,0.11%,2023-10-26,TK_USDJPY BGN Curncy
4,10/25/2023,150.21,149.91,150.32,149.79,NaN,0.20%,2023-10-25,TK_USDJPY BGN Curncy
...,...,...,...,...,...,...,...,...,...
61,08/07/2023,142.49,141.79,142.62,141.52,NaN,0.52%,2023-08-07,TK_USDJPY BGN Curncy
62,08/04/2023,141.75,142.57,142.89,141.55,NaN,-0.55%,2023-08-04,TK_USDJPY BGN Curncy
63,08/03/2023,142.54,143.33,143.90,142.06,NaN,-0.54%,2023-08-03,TK_USDJPY BGN Curncy
64,08/02/2023,143.32,143.34,143.49,142.24,NaN,0.00%,2023-08-02,TK_USDJPY BGN Curncy


In [ ]:
df_S1 = pd.read_csv('US Soybeans Futures Historical Data.csv')
df_S1

,Date,Price,Open,High,Low,Vol.,Change %
0,10/31/2023,"1,310.50","1,307.25","1,321.50","1,300.00",NaN,0.25%
1,10/30/2023,"1,307.25","1,319.50","1,331.25","1,305.50",NaN,-0.93%
2,10/27/2023,"1,319.50","1,300.00","1,325.25","1,298.50",160.76K,3.13%
3,10/26/2023,"1,279.50","1,287.50","1,297.75","1,277.50",NaN,-0.68%
4,10/25/2023,"1,288.25","1,293.00","1,301.25","1,278.00",111.04K,-0.54%
...,...,...,...,...,...,...,...
60,08/07/2023,"1,302.00","1,324.50","1,327.00","1,291.00",124.90K,-2.34%
61,08/04/2023,"1,333.25","1,326.00","1,345.75","1,322.50",86.49K,0.60%
62,08/03/2023,"1,325.25","1,321.50","1,337.25","1,318.50",87.41K,0.30%
63,08/02/2023,"1,321.25","1,344.00","1,354.00","1,315.00",123.79K,-1.49%


In [ ]:
df_S1['date'] = df_S1['Date'].apply(lambda x: pd.to_datetime(str(x), format='%m/%d/%Y')) # 시계열 정보 추출
df_S1

,Date,Price,Open,High,Low,Vol.,Change %,date
0,10/31/2023,"1,310.50","1,307.25","1,321.50","1,300.00",NaN,0.25%,2023-10-31
1,10/30/2023,"1,307.25","1,319.50","1,331.25","1,305.50",NaN,-0.93%,2023-10-30
2,10/27/2023,"1,319.50","1,300.00","1,325.25","1,298.50",160.76K,3.13%,2023-10-27
3,10/26/2023,"1,279.50","1,287.50","1,297.75","1,277.50",NaN,-0.68%,2023-10-26
4,10/25/2023,"1,288.25","1,293.00","1,301.25","1,278.00",111.04K,-0.54%,2023-10-25
...,...,...,...,...,...,...,...,...
60,08/07/2023,"1,302.00","1,324.50","1,327.00","1,291.00",124.90K,-2.34%,2023-08-07
61,08/04/2023,"1,333.25","1,326.00","1,345.75","1,322.50",86.49K,0.60%,2023-08-04
62,08/03/2023,"1,325.25","1,321.50","1,337.25","1,318.50",87.41K,0.30%,2023-08-03
63,08/02/2023,"1,321.25","1,344.00","1,354.00","1,315.00",123.79K,-1.49%,2023-08-02


In [ ]:
df_S1['ticker1'] = 'TK_S 1 Comdty'
df_S1

,Date,Price,Open,High,Low,Vol.,Change %,date,ticker1
0,10/31/2023,"1,310.50","1,307.25","1,321.50","1,300.00",NaN,0.25%,2023-10-31,TK_S 1 Comdty
1,10/30/2023,"1,307.25","1,319.50","1,331.25","1,305.50",NaN,-0.93%,2023-10-30,TK_S 1 Comdty
2,10/27/2023,"1,319.50","1,300.00","1,325.25","1,298.50",160.76K,3.13%,2023-10-27,TK_S 1 Comdty
3,10/26/2023,"1,279.50","1,287.50","1,297.75","1,277.50",NaN,-0.68%,2023-10-26,TK_S 1 Comdty
4,10/25/2023,"1,288.25","1,293.00","1,301.25","1,278.00",111.04K,-0.54%,2023-10-25,TK_S 1 Comdty
...,...,...,...,...,...,...,...,...,...
60,08/07/2023,"1,302.00","1,324.50","1,327.00","1,291.00",124.90K,-2.34%,2023-08-07,TK_S 1 Comdty
61,08/04/2023,"1,333.25","1,326.00","1,345.75","1,322.50",86.49K,0.60%,2023-08-04,TK_S 1 Comdty
62,08/03/2023,"1,325.25","1,321.50","1,337.25","1,318.50",87.41K,0.30%,2023-08-03,TK_S 1 Comdty
63,08/02/2023,"1,321.25","1,344.00","1,354.00","1,315.00",123.79K,-1.49%,2023-08-02,TK_S 1 Comdty


In [ ]:
df_KOSPI = pd.read_csv('KOSPI Historical Data.csv')
df_KOSPI

,Date,Price,Open,High,Low,Vol.,Change %
0,10/31/2023,"2,277.99","2,318.39","2,322.45","2,273.97",394.98M,-1.41%
1,10/30/2023,"2,310.55","2,292.34","2,317.75","2,292.34",444.84M,0.34%
2,10/27/2023,"2,302.81","2,311.59","2,321.84","2,293.05",514.88M,0.16%
3,10/26/2023,"2,299.08","2,325.82","2,330.55","2,299.08",531.07M,-2.71%
4,10/25/2023,"2,363.17","2,389.23","2,389.90","2,362.69",440.81M,-0.85%
5,10/24/2023,"2,383.51","2,371.55","2,384.14","2,326.67",444.16M,1.12%
6,10/23/2023,"2,357.02","2,370.07","2,375.79","2,354.70",395.93M,-0.76%
7,10/20/2023,"2,375.00","2,391.54","2,394.67","2,364.01",521.21M,-1.69%
8,10/19/2023,"2,415.80","2,431.92","2,433.45","2,412.55",756.95M,-1.90%
9,10/18/2023,"2,462.60","2,452.05","2,465.33","2,450.78",911.55M,0.10%


In [ ]:
df_KOSPI['date'] = df_KOSPI['Date'].apply(lambda x: pd.to_datetime(str(x), format='%m/%d/%Y')) # 시계열 정보 추출
df_KOSPI

,Date,Price,Open,High,Low,Vol.,Change %,date
0,10/31/2023,"2,277.99","2,318.39","2,322.45","2,273.97",394.98M,-1.41%,2023-10-31
1,10/30/2023,"2,310.55","2,292.34","2,317.75","2,292.34",444.84M,0.34%,2023-10-30
2,10/27/2023,"2,302.81","2,311.59","2,321.84","2,293.05",514.88M,0.16%,2023-10-27
3,10/26/2023,"2,299.08","2,325.82","2,330.55","2,299.08",531.07M,-2.71%,2023-10-26
4,10/25/2023,"2,363.17","2,389.23","2,389.90","2,362.69",440.81M,-0.85%,2023-10-25
5,10/24/2023,"2,383.51","2,371.55","2,384.14","2,326.67",444.16M,1.12%,2023-10-24
6,10/23/2023,"2,357.02","2,370.07","2,375.79","2,354.70",395.93M,-0.76%,2023-10-23
7,10/20/2023,"2,375.00","2,391.54","2,394.67","2,364.01",521.21M,-1.69%,2023-10-20
8,10/19/2023,"2,415.80","2,431.92","2,433.45","2,412.55",756.95M,-1.90%,2023-10-19
9,10/18/2023,"2,462.60","2,452.05","2,465.33","2,450.78",911.55M,0.10%,2023-10-18


In [ ]:
df_KOSPI['ticker1'] = 'TK_KOSPI Index'
df_KOSPI

,Date,Price,Open,High,Low,Vol.,Change %,date,ticker1
0,10/31/2023,"2,277.99","2,318.39","2,322.45","2,273.97",394.98M,-1.41%,2023-10-31,TK_KOSPI Index
1,10/30/2023,"2,310.55","2,292.34","2,317.75","2,292.34",444.84M,0.34%,2023-10-30,TK_KOSPI Index
2,10/27/2023,"2,302.81","2,311.59","2,321.84","2,293.05",514.88M,0.16%,2023-10-27,TK_KOSPI Index
3,10/26/2023,"2,299.08","2,325.82","2,330.55","2,299.08",531.07M,-2.71%,2023-10-26,TK_KOSPI Index
4,10/25/2023,"2,363.17","2,389.23","2,389.90","2,362.69",440.81M,-0.85%,2023-10-25,TK_KOSPI Index
5,10/24/2023,"2,383.51","2,371.55","2,384.14","2,326.67",444.16M,1.12%,2023-10-24,TK_KOSPI Index
6,10/23/2023,"2,357.02","2,370.07","2,375.79","2,354.70",395.93M,-0.76%,2023-10-23,TK_KOSPI Index
7,10/20/2023,"2,375.00","2,391.54","2,394.67","2,364.01",521.21M,-1.69%,2023-10-20,TK_KOSPI Index
8,10/19/2023,"2,415.80","2,431.92","2,433.45","2,412.55",756.95M,-1.90%,2023-10-19,TK_KOSPI Index
9,10/18/2023,"2,462.60","2,452.05","2,465.33","2,450.78",911.55M,0.10%,2023-10-18,TK_KOSPI Index


In [ ]:
df_SPX = pd.read_csv('S&P 500 Historical Data.csv')
df_SPX

,Date,Price,Open,High,Low,Vol.,Change %
0,10/31/2023,"4,193.80","4,171.33","4,195.55","4,153.12",NaN,0.65%
1,10/30/2023,"4,166.82","4,139.39","4,177.47","4,132.94",NaN,1.20%
2,10/27/2023,"4,117.37","4,152.93","4,156.70","4,103.78",NaN,-0.48%
3,10/26/2023,"4,137.23","4,175.99","4,183.60","4,127.90",NaN,-1.18%
4,10/25/2023,"4,186.77","4,232.42","4,232.42","4,181.42",NaN,-1.43%
...,...,...,...,...,...,...,...
60,08/07/2023,"4,518.44","4,491.58","4,519.84","4,491.15",NaN,0.90%
61,08/04/2023,"4,478.03","4,513.96","4,540.34","4,474.55",NaN,-0.53%
62,08/03/2023,"4,501.89","4,494.27","4,519.49","4,485.54",NaN,-0.25%
63,08/02/2023,"4,513.39","4,550.93","4,550.93","4,505.75",NaN,-1.38%


In [ ]:
df_SPX['date'] = df_SPX['Date'].apply(lambda x: pd.to_datetime(str(x), format='%m/%d/%Y')) # 시계열 정보 추출
df_SPX

,Date,Price,Open,High,Low,Vol.,Change %,date
0,10/31/2023,"4,193.80","4,171.33","4,195.55","4,153.12",NaN,0.65%,2023-10-31
1,10/30/2023,"4,166.82","4,139.39","4,177.47","4,132.94",NaN,1.20%,2023-10-30
2,10/27/2023,"4,117.37","4,152.93","4,156.70","4,103.78",NaN,-0.48%,2023-10-27
3,10/26/2023,"4,137.23","4,175.99","4,183.60","4,127.90",NaN,-1.18%,2023-10-26
4,10/25/2023,"4,186.77","4,232.42","4,232.42","4,181.42",NaN,-1.43%,2023-10-25
...,...,...,...,...,...,...,...,...
60,08/07/2023,"4,518.44","4,491.58","4,519.84","4,491.15",NaN,0.90%,2023-08-07
61,08/04/2023,"4,478.03","4,513.96","4,540.34","4,474.55",NaN,-0.53%,2023-08-04
62,08/03/2023,"4,501.89","4,494.27","4,519.49","4,485.54",NaN,-0.25%,2023-08-03
63,08/02/2023,"4,513.39","4,550.93","4,550.93","4,505.75",NaN,-1.38%,2023-08-02


In [ ]:
df_SPX['ticker1'] = 'TK_SPX Index'
df_SPX

,Date,Price,Open,High,Low,Vol.,Change %,date,ticker1
0,10/31/2023,"4,193.80","4,171.33","4,195.55","4,153.12",NaN,0.65%,2023-10-31,TK_SPX Index
1,10/30/2023,"4,166.82","4,139.39","4,177.47","4,132.94",NaN,1.20%,2023-10-30,TK_SPX Index
2,10/27/2023,"4,117.37","4,152.93","4,156.70","4,103.78",NaN,-0.48%,2023-10-27,TK_SPX Index
3,10/26/2023,"4,137.23","4,175.99","4,183.60","4,127.90",NaN,-1.18%,2023-10-26,TK_SPX Index
4,10/25/2023,"4,186.77","4,232.42","4,232.42","4,181.42",NaN,-1.43%,2023-10-25,TK_SPX Index
...,...,...,...,...,...,...,...,...,...
60,08/07/2023,"4,518.44","4,491.58","4,519.84","4,491.15",NaN,0.90%,2023-08-07,TK_SPX Index
61,08/04/2023,"4,478.03","4,513.96","4,540.34","4,474.55",NaN,-0.53%,2023-08-04,TK_SPX Index
62,08/03/2023,"4,501.89","4,494.27","4,519.49","4,485.54",NaN,-0.25%,2023-08-03,TK_SPX Index
63,08/02/2023,"4,513.39","4,550.93","4,550.93","4,505.75",NaN,-1.38%,2023-08-02,TK_SPX Index


In [ ]:
df_GC1 = pd.read_csv('Gold Futures Historical Data.csv')
df_GC1

,Date,Price,Open,High,Low,Vol.,Change %
0,10/31/2023,"1,994.30","2,005.60","2,017.70","1,987.40",214.78K,-0.56%
1,10/30/2023,"2,005.60","2,013.50","2,016.80","1,999.70",181.34K,0.83%
2,10/27/2023,"1,989.00","1,987.50","2,009.80","1,978.50",0.37K,0.06%
3,10/26/2023,"1,987.90","1,981.60","1,992.50","1,974.40",0.20K,0.12%
4,10/25/2023,"1,985.50","1,972.00","1,988.80","1,971.00",0.16K,0.44%
...,...,...,...,...,...,...,...
62,08/07/2023,"1,970.00","1,977.60","1,981.70","1,966.10",104.41K,-0.31%
63,08/04/2023,"1,976.10","1,969.60","1,984.20","1,954.50",156.42K,0.37%
64,08/03/2023,"1,968.80","1,970.80","1,974.50","1,964.50",141.35K,-0.31%
65,08/02/2023,"1,975.00","1,988.90","1,992.20","1,969.10",169.66K,-0.19%


In [ ]:
df_GC1['date'] = df_GC1['Date'].apply(lambda x: pd.to_datetime(str(x), format='%m/%d/%Y')) # 시계열 정보 추출
df_GC1

,Date,Price,Open,High,Low,Vol.,Change %,date
0,10/31/2023,"1,994.30","2,005.60","2,017.70","1,987.40",214.78K,-0.56%,2023-10-31
1,10/30/2023,"2,005.60","2,013.50","2,016.80","1,999.70",181.34K,0.83%,2023-10-30
2,10/27/2023,"1,989.00","1,987.50","2,009.80","1,978.50",0.37K,0.06%,2023-10-27
3,10/26/2023,"1,987.90","1,981.60","1,992.50","1,974.40",0.20K,0.12%,2023-10-26
4,10/25/2023,"1,985.50","1,972.00","1,988.80","1,971.00",0.16K,0.44%,2023-10-25
...,...,...,...,...,...,...,...,...
62,08/07/2023,"1,970.00","1,977.60","1,981.70","1,966.10",104.41K,-0.31%,2023-08-07
63,08/04/2023,"1,976.10","1,969.60","1,984.20","1,954.50",156.42K,0.37%,2023-08-04
64,08/03/2023,"1,968.80","1,970.80","1,974.50","1,964.50",141.35K,-0.31%,2023-08-03
65,08/02/2023,"1,975.00","1,988.90","1,992.20","1,969.10",169.66K,-0.19%,2023-08-02


In [ ]:
df_GC1['ticker1'] = 'TK_GC1 Comdty'
df_GC1

,Date,Price,Open,High,Low,Vol.,Change %,date,ticker1
0,10/31/2023,"1,994.30","2,005.60","2,017.70","1,987.40",214.78K,-0.56%,2023-10-31,TK_GC1 Comdty
1,10/30/2023,"2,005.60","2,013.50","2,016.80","1,999.70",181.34K,0.83%,2023-10-30,TK_GC1 Comdty
2,10/27/2023,"1,989.00","1,987.50","2,009.80","1,978.50",0.37K,0.06%,2023-10-27,TK_GC1 Comdty
3,10/26/2023,"1,987.90","1,981.60","1,992.50","1,974.40",0.20K,0.12%,2023-10-26,TK_GC1 Comdty
4,10/25/2023,"1,985.50","1,972.00","1,988.80","1,971.00",0.16K,0.44%,2023-10-25,TK_GC1 Comdty
...,...,...,...,...,...,...,...,...,...
62,08/07/2023,"1,970.00","1,977.60","1,981.70","1,966.10",104.41K,-0.31%,2023-08-07,TK_GC1 Comdty
63,08/04/2023,"1,976.10","1,969.60","1,984.20","1,954.50",156.42K,0.37%,2023-08-04,TK_GC1 Comdty
64,08/03/2023,"1,968.80","1,970.80","1,974.50","1,964.50",141.35K,-0.31%,2023-08-03,TK_GC1 Comdty
65,08/02/2023,"1,975.00","1,988.90","1,992.20","1,969.10",169.66K,-0.19%,2023-08-02,TK_GC1 Comdty


In [ ]:
df = pd.concat([df_KOSPI, df_HG1, df_GC1, df_CL1, df_S1, df_SPX, USDJPY, USDKRW])
df

,Date,Price,Open,High,Low,Vol.,Change %,date,ticker1
0,10/31/2023,"2,277.99","2,318.39","2,322.45","2,273.97",394.98M,-1.41%,2023-10-31,TK_KOSPI Index
1,10/30/2023,"2,310.55","2,292.34","2,317.75","2,292.34",444.84M,0.34%,2023-10-30,TK_KOSPI Index
2,10/27/2023,"2,302.81","2,311.59","2,321.84","2,293.05",514.88M,0.16%,2023-10-27,TK_KOSPI Index
3,10/26/2023,"2,299.08","2,325.82","2,330.55","2,299.08",531.07M,-2.71%,2023-10-26,TK_KOSPI Index
4,10/25/2023,"2,363.17","2,389.23","2,389.90","2,362.69",440.81M,-0.85%,2023-10-25,TK_KOSPI Index
...,...,...,...,...,...,...,...,...,...
61,08/07/2023,"1,307.30","1,304.29","1,309.06","1,298.80",NaN,0.25%,2023-08-07,TK_USDKRW BGN Curncy
62,08/04/2023,"1,304.08","1,299.93","1,312.38","1,296.82",NaN,0.39%,2023-08-04,TK_USDKRW BGN Curncy
63,08/03/2023,"1,299.01","1,299.47","1,304.23","1,294.55",NaN,0.01%,2023-08-03,TK_USDKRW BGN Curncy
64,08/02/2023,"1,298.93","1,289.40","1,303.09","1,286.94",NaN,0.77%,2023-08-02,TK_USDKRW BGN Curncy


In [ ]:
df_pivot = pd.pivot(df, index='date', columns='ticker1', values='Price') # index를 date로 ticker1을 컬럼으로 값을 수치로 dataframe을 새로 구성하겠다.
df_pivot.head(50)

ticker1,TK_CL1 Comdty,TK_GC1 Comdty,TK_HG1 Comdty,TK_KOSPI Index,TK_S 1 Comdty,TK_SPX Index,TK_USDJPY BGN Curncy,TK_USDKRW BGN Curncy
date,,,,,,,,
2023-08-01,81.37,"1,978.80",3.9085,"2,667.07","1,341.25","4,576.73",143.32,"1,289.00"
2023-08-02,79.49,"1,975.00",3.8435,"2,616.47","1,321.25","4,513.39",143.32,"1,298.93"
2023-08-03,81.55,"1,968.80",3.8995,"2,605.39","1,325.25","4,501.89",142.54,"1,299.01"
2023-08-04,82.82,"1,976.10",3.8675,"2,602.80","1,333.25","4,478.03",141.75,"1,304.08"
2023-08-07,81.94,"1,970.00",3.835,"2,580.71","1,302.00","4,518.44",142.49,"1,307.30"
2023-08-08,82.92,"1,959.90",3.7665,"2,573.98","1,306.00","4,499.38",143.37,"1,316.72"
2023-08-09,84.4,"1,950.60",3.7835,"2,605.12","1,308.50","4,467.71",143.72,"1,313.08"
2023-08-10,82.82,"1,948.90",3.765,"2,601.56","1,318.25","4,468.83",144.73,"1,316.90"
2023-08-11,83.19,"1,946.60",3.719,"2,591.26","1,307.50","4,464.05",144.96,"1,330.30"


In [ ]:
# 각 월의 첫째날 값으로 resampling
beginning_price = df_pivot.resample('M').first()
beginning_price

ticker1,TK_CL1 Comdty,TK_GC1 Comdty,TK_HG1 Comdty,TK_KOSPI Index,TK_S 1 Comdty,TK_SPX Index,TK_USDJPY BGN Curncy,TK_USDKRW BGN Curncy
date,,,,,,,,
2023-08-31,81.37,"1,978.80",3.9085,"2,667.07","1,341.25","4,576.73",143.32,"1,289.00"
2023-09-30,85.55,"1,967.10",3.8520,"2,563.71","1,356.25","4,515.77",146.23,"1,317.55"
2023-10-31,88.82,"1,847.20",3.6415,"2,405.69","1,277.00","4,288.39",149.85,"1,355.84"


In [ ]:
beginning_price['TK_GC1 Comdty'] = [1978.80, 1967.10, 1847.20]
beginning_price['TK_KOSPI Index'] = [2667.07, 2563.71, 2405.69]
beginning_price['TK_S 1 Comdty'] = [1341.25, 1356.25, 1277.00]
beginning_price['TK_SPX Index'] = [4576.73, 4515.77, 4288.39]
beginning_price['TK_USDKRW BGN Curncy'] = [1289.00, 1317.55, 1355.84]
beginning_price['TK_LT12TRUU Index'] = [287.78, 289.10, 290.34]
beginning_price['TK_LUTLTRUU Index'] = [3252.24, 3164.69, 2930.86]
beginning_price.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3 entries, 2023-08-31 to 2023-10-31
Freq: M
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   TK_CL1 Comdty         3 non-null      float64
 1   TK_GC1 Comdty         3 non-null      float64
 2   TK_HG1 Comdty         3 non-null      float64
 3   TK_KOSPI Index        3 non-null      float64
 4   TK_S 1 Comdty         3 non-null      float64
 5   TK_SPX Index          3 non-null      float64
 6   TK_USDJPY BGN Curncy  3 non-null      float64
 7   TK_USDKRW BGN Curncy  3 non-null      float64
 8   TK_LT12TRUU Index     3 non-null      float64
 9   TK_LUTLTRUU Index     3 non-null      float64
dtypes: float64(10)
memory usage: 264.0 bytes


In [ ]:
# 각 월의 마지막날 값으로 resampling
end_price = df_pivot.resample('M').last()
end_price

ticker1,TK_CL1 Comdty,TK_GC1 Comdty,TK_HG1 Comdty,TK_KOSPI Index,TK_S 1 Comdty,TK_SPX Index,TK_USDJPY BGN Curncy,TK_USDKRW BGN Curncy
date,,,,,,,,
2023-08-31,83.63,"1,965.90",3.8220,"2,556.27","1,360.00","4,507.66",145.53,"1,325.14"
2023-09-30,90.79,"1,866.10",3.7375,"2,465.07","1,275.00","4,288.05",149.35,"1,352.31"
2023-10-31,81.02,"1,994.30",3.6490,"2,277.99","1,310.50","4,193.80",151.67,"1,352.15"


In [ ]:
end_price['TK_GC1 Comdty'] = [1965.90, 1866.10, 1994.30]
end_price['TK_KOSPI Index'] = [2556.27, 2465.07, 2277.99]
end_price['TK_S 1 Comdty'] = [1360.00, 1275.00, 1310.50]
end_price['TK_SPX Index'] = [4507.66, 4288.05, 4193.80]
end_price['TK_USDKRW BGN Curncy'] = [1325.14, 1352.31, 1352.15]
end_price['TK_LT12TRUU Index'] = [289.02, 290.28, 291.61]
end_price['TK_LUTLTRUU Index'] = [3210.40, 2976.22, 2829.35]
end_price.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3 entries, 2023-08-31 to 2023-10-31
Freq: M
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   TK_CL1 Comdty         3 non-null      float64
 1   TK_GC1 Comdty         3 non-null      float64
 2   TK_HG1 Comdty         3 non-null      float64
 3   TK_KOSPI Index        3 non-null      float64
 4   TK_S 1 Comdty         3 non-null      float64
 5   TK_SPX Index          3 non-null      float64
 6   TK_USDJPY BGN Curncy  3 non-null      float64
 7   TK_USDKRW BGN Curncy  3 non-null      float64
 8   TK_LT12TRUU Index     3 non-null      float64
 9   TK_LUTLTRUU Index     3 non-null      float64
dtypes: float64(10)
memory usage: 264.0 bytes


In [ ]:
# 월 수익률화
diff = (beginning_price - end_price)
monthly_return = diff.div(beginning_price)
monthly_return

ticker1,TK_CL1 Comdty,TK_GC1 Comdty,TK_HG1 Comdty,TK_KOSPI Index,TK_S 1 Comdty,TK_SPX Index,TK_USDJPY BGN Curncy,TK_USDKRW BGN Curncy,TK_LT12TRUU Index,TK_LUTLTRUU Index
date,,,,,,,,,,
2023-08-31,-0.027774,0.006519,0.022131,0.041544,-0.013979,0.015092,-0.015420,-0.028037,-0.004309,0.012865
2023-09-30,-0.061251,0.051345,0.029725,0.038475,0.059908,0.050428,-0.021336,-0.026382,-0.004082,0.059554
2023-10-31,0.087818,-0.079634,-0.002060,0.053082,-0.026233,0.022057,-0.012145,0.002722,-0.004374,0.034635


In [ ]:
monthly_return.to_csv(
    path_or_buf = '/content/drive/MyDrive/project/df_pred.csv',
    index = True,
    sep = ',',      # 구분기호 설정
    encoding = 'utf-8',
    float_format='%f',     # 숫자 자리수 모두 표시
)

In [ ]:
df_cut = pd.read_csv('df_train_set.csv', index_col='date', parse_dates=True)
df_cut

,TK_CL1 Comdty,TK_DAX Index,TK_DXY Curncy,TK_GC1 Comdty,TK_HG1 Comdty,TK_KOSPI Index,TK_NKY Index,TK_S 1 Comdty,TK_SPX Index,TK_USDJPY BGN Curncy,TK_USDKRW BGN Curncy,TK_XAU Index,국면,TK_LT12TRUU Index,TK_LUTLTRUU Index
date,,,,,,,,,,,,,,,
1989-01-31,-0.019009,-0.016689,0.049463,-0.052733,0.036893,-0.038408,0.044229,-0.036327,0.080491,0.043583,-0.005819,0.053540,0,0.005066,-0.020756
1989-02-28,0.036551,-0.009119,-0.019525,-0.008382,-0.002872,0.035210,0.019927,-0.006516,-0.027702,-0.021065,-0.011266,0.115738,0,0.006622,0.010665
1989-03-31,0.104486,0.011788,0.026654,-0.005167,-0.087154,0.094910,0.027355,-0.016242,0.027028,0.036456,-0.001776,-0.040843,0,0.010309,0.025407
1989-04-30,0.023559,0.019749,0.004420,-0.014759,0.099309,-0.066712,0.020316,0.017430,0.044705,0.006058,-0.007125,-0.049984,0,0.008941,0.036788
1989-05-31,-0.036786,0.032843,0.048604,-0.045705,-0.158178,-0.014204,0.014014,-0.028819,0.036879,0.061610,0.000598,-0.025203,0,0.010114,0.057119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-30,0.002410,0.039678,0.004661,-0.054143,0.026269,0.047664,0.055075,0.057793,0.024248,0.017319,-0.011901,-0.117853,3,0.001832,-0.021883
2019-10-31,0.010444,0.049166,-0.017916,0.022132,0.030268,0.005337,0.047603,-0.002991,0.033096,0.002599,-0.029546,0.079964,3,0.002267,-0.017879
2019-11-30,-0.018327,0.021243,0.010634,-0.030303,-0.004146,-0.005828,0.019393,-0.051393,0.024151,0.012016,0.013650,-0.004378,3,0.001167,-0.013492


In [ ]:
df_cut.drop(columns=['TK_DAX Index', 'TK_DXY Curncy', 'TK_NKY Index','TK_XAU Index'],inplace=True)
df_cut

,TK_CL1 Comdty,TK_GC1 Comdty,TK_HG1 Comdty,TK_KOSPI Index,TK_S 1 Comdty,TK_SPX Index,TK_USDJPY BGN Curncy,TK_USDKRW BGN Curncy,국면,TK_LT12TRUU Index,TK_LUTLTRUU Index
date,,,,,,,,,,,
1989-01-31,-0.019009,-0.052733,0.036893,-0.038408,-0.036327,0.080491,0.043583,-0.005819,0,0.005066,-0.020756
1989-02-28,0.036551,-0.008382,-0.002872,0.035210,-0.006516,-0.027702,-0.021065,-0.011266,0,0.006622,0.010665
1989-03-31,0.104486,-0.005167,-0.087154,0.094910,-0.016242,0.027028,0.036456,-0.001776,0,0.010309,0.025407
1989-04-30,0.023559,-0.014759,0.099309,-0.066712,0.017430,0.044705,0.006058,-0.007125,0,0.008941,0.036788
1989-05-31,-0.036786,-0.045705,-0.158178,-0.014204,-0.028819,0.036879,0.061610,0.000598,0,0.010114,0.057119
...,...,...,...,...,...,...,...,...,...,...,...
2019-09-30,0.002410,-0.054143,0.026269,0.047664,0.057793,0.024248,0.017319,-0.011901,3,0.001832,-0.021883
2019-10-31,0.010444,0.022132,0.030268,0.005337,-0.002991,0.033096,0.002599,-0.029546,3,0.002267,-0.017879
2019-11-30,-0.018327,-0.030303,-0.004146,-0.005828,-0.051393,0.024151,0.012016,0.013650,3,0.001167,-0.013492


In [ ]:
# features/target, train/test dataset 분리
# 다운 될 때 대비용
from sklearn.model_selection import train_test_split

feature_columns = list(df_cut.columns.difference(['국면']))
X = df_cut[feature_columns]
y = df_cut['국면']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)#, random_state = 42) # 학습데이터와 평가데이터의 비율을 8:2 로 분할|
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape) # 데이터 개수 확인

(298, 10) (75, 10) (298,) (75,)


In [ ]:
# 순서 맞춰주기
df_cut.columns

Index(['TK_CL1 Comdty', 'TK_GC1 Comdty', 'TK_HG1 Comdty', 'TK_KOSPI Index',
       'TK_S 1 Comdty', 'TK_SPX Index', 'TK_USDJPY BGN Curncy',
       'TK_USDKRW BGN Curncy', '국면', 'TK_LT12TRUU Index', 'TK_LUTLTRUU Index'],
      dtype='object')

In [ ]:
monthly_return.columns

Index(['TK_CL1 Comdty', 'TK_GC1 Comdty', 'TK_HG1 Comdty', 'TK_KOSPI Index',
       'TK_S 1 Comdty', 'TK_SPX Index', 'TK_USDJPY BGN Curncy',
       'TK_USDKRW BGN Curncy', 'TK_LT12TRUU Index', 'TK_LUTLTRUU Index'],
      dtype='object', name='ticker1')

In [ ]:
monthly_return = pd.read_csv('df_pred.csv', index_col='date', parse_dates=True)
monthly_return

,TK_CL1 Comdty,TK_GC1 Comdty,TK_HG1 Comdty,TK_KOSPI Index,TK_S 1 Comdty,TK_SPX Index,TK_USDJPY BGN Curncy,TK_USDKRW BGN Curncy,TK_LT12TRUU Index,TK_LUTLTRUU Index
date,,,,,,,,,,
2023-08-31,-0.027774,0.006519,0.022131,0.041544,-0.013979,0.015092,-0.015420,-0.028037,-0.004309,0.012865
2023-09-30,-0.061251,0.051345,0.029725,0.038475,0.059908,0.050428,-0.021336,-0.026382,-0.004082,0.059554
2023-10-31,0.087818,-0.079634,-0.002060,0.053082,-0.026233,0.022057,-0.012145,0.002722,-0.004374,0.034635


In [ ]:
monthly_return['국면'] = [0,0,0]

In [ ]:
monthly_return = monthly_return[['TK_CL1 Comdty', 'TK_GC1 Comdty', 'TK_HG1 Comdty', 'TK_KOSPI Index',
       'TK_LT12TRUU Index', 'TK_LUTLTRUU Index', 'TK_S 1 Comdty',
       'TK_SPX Index', 'TK_USDJPY BGN Curncy', 'TK_USDKRW BGN Curncy']]

In [ ]:
monthly_return = monthly_return.drop(columns='국면', inplace=True)

KeyError: ignored

In [ ]:
monthly_return

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape)

(298, 10) (3, 10) (298,)


In [ ]:
# 예측값 확인
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=500, max_depth=10,random_state=0,min_samples_leaf=3, min_samples_split= 2)
clf.fit(X_train,y_train)
X_test = monthly_return

predict2 = clf.predict(X_test)
predict2

array([3, 3, 3])

In [ ]:
X_train

,TK_CL1 Comdty,TK_GC1 Comdty,TK_HG1 Comdty,TK_KOSPI Index,TK_LT12TRUU Index,TK_LUTLTRUU Index,TK_S 1 Comdty,TK_SPX Index,TK_USDJPY BGN Curncy,TK_USDKRW BGN Curncy
date,,,,,,,,,,
2005-01-31,0.144349,-0.018385,-0.026128,0.043627,0.001594,0.024231,-0.041880,-0.017312,0.008951,-0.011656
1995-04-30,0.070941,-0.010695,-0.066835,-0.039707,0.006145,0.010245,-0.026191,0.025625,-0.017589,-0.013665
1996-01-31,-0.104493,0.034958,-0.027942,-0.011284,0.003391,-0.004164,-0.013026,0.024632,0.034799,0.014282
2019-02-28,0.035469,-0.000607,0.063830,-0.003640,0.001860,-0.013519,-0.022065,0.028804,0.017260,0.004853
1990-05-31,-0.078390,-0.022538,-0.092138,0.110655,0.007255,0.023010,-0.032078,0.087223,-0.039093,0.008319
...,...,...,...,...,...,...,...,...,...,...
1994-12-31,-0.003367,0.015320,-0.001801,-0.036428,0.006217,0.014669,-0.010777,0.023055,0.002012,-0.006192
1997-11-30,-0.086355,-0.055662,-0.070757,-0.179719,0.003913,0.020258,0.001743,0.017476,0.053130,0.202123
2011-07-31,0.008005,0.098273,0.043686,0.003514,0.000000,0.055672,0.024201,-0.035374,-0.050353,-0.011794


In [ ]:
X_test.columns

Index(['TK_CL1 Comdty', 'TK_GC1 Comdty', 'TK_HG1 Comdty', 'TK_KOSPI Index',
       'TK_LT12TRUU Index', 'TK_LUTLTRUU Index', 'TK_S 1 Comdty',
       'TK_SPX Index', 'TK_USDJPY BGN Curncy', 'TK_USDKRW BGN Curncy'],
      dtype='object')

In [ ]:
# features/target, train/test dataset 분리
# 다운 될 때 대비용
# df_cut = pd.read_csv('df_train_set.csv', index_col='date', parse_dates=True)

feature_columns = list(df_cut.columns.difference(['국면']))
X = df_cut[feature_columns]
y = df_cut['국면']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)#, random_state = 42) # 학습데이터와 평가데이터의 비율을 8:2 로 분할|
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape) # 데이터 개수 확인

(298, 10) (75, 10) (298,) (75,)


In [ ]:
#기본적인 randomforest모형

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score # 정확도 함수

clf = RandomForestClassifier(random_state=0)
clf.fit(X_train,y_train)

predict1 = clf.predict(X_test)
print(accuracy_score(y_test,predict1))

0.6533333333333333


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=500, max_depth=10,random_state=0,min_samples_leaf=3, min_samples_split= 2)
clf.fit(X_train,y_train)

predict2 = clf.predict(X_test)
print(accuracy_score(y_test,predict2))

0.6533333333333333
